In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Hopfield:
    def __init__(self, n):
        self.n = n
        self.W = np.zeros((n, n), dtype=np.int64)
        self.P = 0

    def get_W(self, patterns):
        P = np.dot(patterns.T, patterns)
        np.fill_diagonal(P, 0.0)
        self.W = P

    def store(self, X):
        self.get_W(X)
        self.P = X.shape[1]
        return self.W

    def check_pattern(self, pattern, epochs=5, report=True):
        stable = False
        a = np.array(pattern)
        A = [a]
        D = [math.inf]

        if report:
            print("Pattern:", pattern)
            
        for i in range(epochs):
            u = np.dot(a, self.W)
            a = np.sign(u)
            d = self.P - np.count_nonzero(np.equal(pattern, a)) # difference
            A.append(a)
            D.append(d)
            
            if np.array_equal(A[-1], A[-2]): # if converged
                if i == 0:
                    stable = True
                    if report:
                        print("Pattern is stable.")
                elif report:
                    print("converged to same pattern in iteration:", i)
                break

        A = np.array(A)
        D = np.array(D)

        min_ind = np.argmin(D)
        nearest = A[min_ind]
        accuracy = (self.P - D[min_ind]) / self.P
        energy = -1 * np.sum(np.multiply(np.outer(nearest, nearest), self.W))
        
        if report:
            print("Nearest pattern found in iteration:", min_ind)
            print("Nearest pattern:", nearest)
            print("Accuracy:", accuracy * 100, "%")
            print("Energy:", energy)

        return stable, nearest, accuracy, energy

    def find_stable_patterns(self):

        stable_patterns = []
        for i in range(2 ** self.n):

            binary = list(format(i, f'0{self.n}b'))
            pattern = [1 if int(bit) else -1 for bit in binary]
            stable = self.check_pattern(pattern, report=False)[0]

            if(stable):
                stable_patterns.append(pattern)

        return stable_patterns

In [3]:
hopfield = Hopfield(8)

X = np.array([
    [-1, -1, 1, -1, 1, -1, -1, 1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, 1, 1, -1, -1, 1, -1, 1]
])

W = hopfield.store(X)

print(X.shape[0], 'patterns stored.\n')
print('Weight Matrix:\n\n', W)

3 patterns stored.

Weight Matrix:

 [[ 0  1 -1  3  1 -1  3 -1]
 [ 1  0  1  1 -1  1  1  1]
 [-1  1  0 -1  1 -1 -1  3]
 [ 3  1 -1  0  1 -1  3 -1]
 [ 1 -1  1  1  0 -3  1  1]
 [-1  1 -1 -1 -3  0 -1 -1]
 [ 3  1 -1  3  1 -1  0 -1]
 [-1  1  3 -1  1 -1 -1  0]]


## 1.1 check X1, X2, X3 patterns

In [4]:
X1 = X[0]
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X1)

Pattern: [-1 -1  1 -1  1 -1 -1  1]
Pattern is stable.
Nearest pattern found in iteration: 1
Nearest pattern: [-1 -1  1 -1  1 -1 -1  1]
Accuracy: 100.0 %
Energy: -44


In [5]:
X2 = X[1]
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X2)

Pattern: [-1 -1 -1 -1 -1  1 -1 -1]
Pattern is stable.
Nearest pattern found in iteration: 1
Nearest pattern: [-1 -1 -1 -1 -1  1 -1 -1]
Accuracy: 100.0 %
Energy: -44


In [6]:
X3 = X[2]
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X3)

Pattern: [-1  1  1 -1 -1  1 -1  1]
Pattern is stable.
Nearest pattern found in iteration: 1
Nearest pattern: [-1  1  1 -1 -1  1 -1  1]
Accuracy: 100.0 %
Energy: -48


## 1.2 check X1n, X2n, X3n patterns

In [7]:
X1n = [1, -1, 1, -1, 1, -1, -1, 1]
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X1n)

Pattern: [1, -1, 1, -1, 1, -1, -1, 1]
converged to same pattern in iteration: 1
Nearest pattern found in iteration: 1
Nearest pattern: [-1 -1  1 -1  1 -1 -1  1]
Accuracy: 87.5 %
Energy: -44


In [8]:
X2n = [1, 1, -1, -1, -1, 1, -1, -1]
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X2n)

Pattern: [1, 1, -1, -1, -1, 1, -1, -1]
Nearest pattern found in iteration: 2
Nearest pattern: [ 1  1 -1 -1 -1  1 -1 -1]
Accuracy: 100.0 %
Energy: -12


In [9]:
X3n = [1, 1, 1, -1, 1, 1, -1, 1]
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X3n)

Pattern: [1, 1, 1, -1, 1, 1, -1, 1]
Nearest pattern found in iteration: 2
Nearest pattern: [-1 -1  1 -1  1  1 -1  1]
Accuracy: 75.0 %
Energy: -32


## 1.3 Stable Patterns

In [10]:
stable_patterns = hopfield.find_stable_patterns()
print('stable patterns count:', len(stable_patterns))
print('these patterns are stable:\n')
print(*stable_patterns, sep='\n')

stable patterns count: 6
these patterns are stable:

[-1, -1, -1, -1, -1, 1, -1, -1]
[-1, -1, 1, -1, 1, -1, -1, 1]
[-1, 1, 1, -1, -1, 1, -1, 1]
[1, -1, -1, 1, 1, -1, 1, -1]
[1, 1, -1, 1, -1, 1, 1, -1]
[1, 1, 1, 1, 1, -1, 1, 1]


## 1.4 Noisy Patterns

In [11]:
X3nn = [-1, 1, -1, 1, 1, -1, 1, -1] #6 bit noise
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X3nn, 10)

Pattern: [-1, 1, -1, 1, 1, -1, 1, -1]
converged to same pattern in iteration: 1
Nearest pattern found in iteration: 1
Nearest pattern: [ 1 -1 -1  1  1 -1  1 -1]
Accuracy: 75.0 %
Energy: -48


In [12]:
X4nn = [1, -1, 1, -1, 1, -1, -1, -1] #4 bit noise
stable, nearest_pattern, accuracy, energy = hopfield.check_pattern(X4nn, 10)

Pattern: [1, -1, 1, -1, 1, -1, -1, -1]
Nearest pattern found in iteration: 2
Nearest pattern: [ 1 -1  1  1  1 -1  1 -1]
Accuracy: 75.0 %
Energy: -28
